In [2]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA"

## 1. GSEA analysis for each set of comparison

In [3]:
##########----- Parameters-----##########
genome <- "mm"
#########################################
sum.dir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_shRNA_RNA/2_DE_GSEA'
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)

###----- Find all de files
de.out.dir1 <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_shRNA_RNA/1_DEseq2_merge_shChd7'
de.out.dir2 <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_shRNA_RNA/1_DEseq2'
de.files1 <- list.files(de.out.dir1, pattern="*gn.csv", full.names=TRUE, recursive=TRUE)
de.files2 <- list.files(de.out.dir2, pattern="*gn.csv", full.names=TRUE, recursive=TRUE)
de.files <- c(de.files1, de.files2)

use.cols <- c("gene_name", "log2FoldChange", "padj")

###----- Select gene signature
gs.dir <- file.path('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm')
gs.pattern <- paste("_", genome, "_sigs.csv", sep="")
gs.files <- list.files(gs.dir, pattern=gs.pattern, full.names=TRUE)
gs.files <- c(gs.files,'/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv',
              '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/1_IL2RAKO/1_Scanpy/0-all_Scanpy_out/2_DE/D6-5_IL2_dependent_mm_sigs.csv')

# gs.files
gs.files <- c('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/2020_mouse_CD8_mm_sigs.csv',
'/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Exp391_CD8_mm_sigs.csv',
'/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv',
'/media/pipkin/ROCKET-PRO/CD8_DEV_SC/1_IL2RAKO/1_Scanpy/0-all_Scanpy_out/2_DE/D6-5_IL2_dependent_mm_sigs.csv')

gs.files
fc.cutoff <- 1

[1] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/2020_mouse_CD8_mm_sigs.csv"         
[2] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Exp391_CD8_mm_sigs.csv"             
[3] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv"                  
[4] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/1_IL2RAKO/1_Scanpy/0-all_Scanpy_out/2_DE/D6-5_IL2_dependent_mm_sigs.csv"

In [6]:
completed.df <- data.frame()
for (gs.file.i in c("file", gs.files)) {
    gs.file.base <- gsub(gs.pattern, "",basename(gs.file.i))
    completed.df[gs.file.base] <- character(0)
}
completed.name <- file.path(sum.dir, "completed.csv")

for (file.i in de.files) {
    file.i.qc.row <- c(file.i)
    
    ###----- Create output names & output directory
    file.i.outpath <- gsub(de.out.dir1, "", file.i)
    file.i.outpath <- gsub(de.out.dir2, "", file.i.outpath)
    file.i.outpath <- gsub(".csv", "", file.i.outpath)
    file.i.outpath <- gsub("^/", "",file.i.outpath)

    file.i.simp.name <- tail(unlist(strsplit( file.i.outpath, "/")), 1)
    file.i.simp.name <- gsub("_differential", "", file.i.simp.name)
    file.i.simp.vec <- unlist(strsplit(file.i.simp.name, "_vs_"))

    # Create output directory
    file.i.dir <- file.path(sum.dir, paste(file.i.outpath, "_sep", sep=""))
    dir.create(file.i.dir, recursive=TRUE, showWarnings=FALSE)
    
    for (gs.file.i in gs.files) {
        gs.file.base <- gsub(gs.pattern, "",basename(gs.file.i))
        file.i.gs.dir <- file.path(file.i.dir, gs.file.base)
        dir.create(file.i.gs.dir, recursive=TRUE, showWarnings=FALSE)
        setwd(file.i.gs.dir)
        
        # Run GSEA analysis
        useGroup <- "log2FoldChange"
        outName <- file.i.simp.name
        
        success <- "yes"
        gsea <- tryCatch({
            GSEA_analysis(file.i, useGroup, outName, gs.file.i, fc.cutoff)
        }, error=function(cond) {
            message(paste(file.i, gs.file.i,sep='; '))
            return("no")
        })
        
        if (!is.null(gsea)){
            success <- ""
        }
        
        file.i.qc.row <- c(file.i.qc.row, success)
    }
    
    completed.df[nrow(completed.df) +1, ] <- file.i.qc.row
    write.csv(completed.df, completed.name, row.names=TRUE)
}


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


[1] 3615



── Column specification ───────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


[1] 3615



── Column specification ───────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


...


── Column specification ───────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


[1] 5144



── Column specification ───────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ───────────────────────────────────
cols(
  gene_name = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Unknown columns: `gene_names`”


[1] 5144



── Column specification ───────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...

